In [1]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOLavaGridDR-v0" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 2 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

from helpers.utils import ENVIRONMENTS_MAP, get_algorithms
ALGORITHMS = get_algorithms(ENV_NAME)

### Calculate EUM for Generalist and Specialist

We don't normalize for EUM unlike in NHGR. The chosen utility functions should already express the weightage and preference over objectives

Import helpers

In [2]:
sys.path.append(os.path.abspath(os.path.join('../..')))

from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100 # CHANGE THIS TO THE NUMBER OF WEIGHTS YOU WANT TO USE, NORMALLY ITS 100, FOR MARIO ITS 32
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 1.]),
 array([0.01019616, 0.98980384]),
 array([0.02050503, 0.97949497]),
 array([0.03075785, 0.96924215]),
 array([0.04110771, 0.95889229])]

### Combine the fronts of all the Specialists for each environment

Skip this step if already done before

In [3]:
import warnings
from mo_utils.pareto import filter_pareto_dominated

curr_envs = ENVIRONMENTS_MAP[ENV_NAME]
SPECIALIST_FRONT = "eval/front" # don't change this, this is the discounted fronts but poorly named!!
path_to_find_fronts = f"data/single_env/{SPECIALIST_FRONT}/{ENV_NAME}"

for env in curr_envs:
    unfiltered_combined_front_df = None
    path_to_find_front_for_subenv = path_to_find_fronts + f"/{env}"
    
    for algo in ALGORITHMS:
        if os.path.exists(path_to_find_front_for_subenv + f"/{algo}.csv"):
            front_df = pd.read_csv(path_to_find_front_for_subenv + f"/{algo}.csv")
            print(f"Found front for {env} - {algo}, total row: {len(front_df)}")
            if unfiltered_combined_front_df is None:
                unfiltered_combined_front_df = front_df
            else:
                unfiltered_combined_front_df = pd.concat([unfiltered_combined_front_df, front_df])

    if unfiltered_combined_front_df is None:
        warnings.warn(f"No fronts found for {env}")
        continue
    
    unfiltered_combined_front_df = unfiltered_combined_front_df.reset_index(drop=True)
    print(f"Combined front for {env} has {len(unfiltered_combined_front_df)} rows")

    for column in unfiltered_combined_front_df.columns:
        min_value = unfiltered_combined_front_df[column].min()
        max_value = unfiltered_combined_front_df[column].max()
        print(f"{column}, Min: {min_value}, Max: {max_value}")

    combined_front_array = unfiltered_combined_front_df.to_numpy()
    filtered_combined_front_array = filter_pareto_dominated(combined_front_array)

    combined_front_df = pd.DataFrame(filtered_combined_front_array, columns=unfiltered_combined_front_df.columns)
    print(f"Filtered front for {env} has {len(combined_front_df)} rows")
    save_dir = f"data/single_env/combined_fronts/{ENV_NAME}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    combined_front_df.to_csv(f"{save_dir}/{env}.csv", index=False)

Found front for MOLavaGridCheckerBoard-v0 - GPI-LS, total row: 1
Combined front for MOLavaGridCheckerBoard-v0 has 1 rows
objective_1, Min: 107.33586502075195, Max: 107.33586502075195
objective_2, Min: 218.76215887069705, Max: 218.76215887069705
Filtered front for MOLavaGridCheckerBoard-v0 has 1 rows
Found front for MOLavaGridSmiley-v0 - GPI-LS, total row: 3
Combined front for MOLavaGridSmiley-v0 has 3 rows
objective_1, Min: 242.1366844177246, Max: 270.69044494628906
objective_2, Min: 192.12224054336548, Max: 225.50174689292908
Filtered front for MOLavaGridSmiley-v0 has 3 rows
Found front for MOLavaGridSnake-v0 - GPI-LS, total row: 2
Combined front for MOLavaGridSnake-v0 has 2 rows
objective_1, Min: 222.61119079589844, Max: 234.21388816833496
objective_2, Min: 187.73548531532288, Max: 220.5411500930786
Filtered front for MOLavaGridSnake-v0 has 2 rows
Found front for MOLavaGridIslands-v0 - GPI-LS, total row: 2
Combined front for MOLavaGridIslands-v0 has 2 rows
objective_1, Min: 87.679607

In [4]:
from helpers.utils import ENVIRONMENTS_MAP

FRONT = "eval/discounted_front" # don't change this, front extracted for specialists are only the discounted ones!!
file_path = f"data/{FRONT}/{ENV_NAME}"
scores_save_path = f"data/eugr_scores/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

In [5]:
specialist_eums = []

for env in ENVIRONMENTS_MAP[ENV_NAME]:
    best_env_front_path = f"data/single_env/combined_fronts/{ENV_NAME}/{env}.csv"
    assert os.path.exists(best_env_front_path), f"File {best_env_front_path} does not exist"
    
    best_env_front = pd.read_csv(best_env_front_path)
    data_array = best_env_front.to_numpy()
    specialist_eums.append(expected_utility(data_array, weights_set=EVAL_WEIGHTS))

specialist_data = {f"eum/{env}": [specialist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
specialist_eum = pd.DataFrame(specialist_data)
specialist_eum.to_csv(f"{scores_save_path}/specialist.csv", index=False)

### Calculate EUM for GENERALIST

In [6]:
# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        generalist_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()

            generalist_eums.append(expected_utility(data_array, weights_set=EVAL_WEIGHTS))

        data = {f"eum/{env}": [generalist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)
            

### Calculate EUGR

In [7]:
# get the normalized hypervolumes of the specialists
specialist_eum_data = pd.read_csv(f"{scores_save_path}/specialist.csv")

for algo in ALGORITHMS:
    for seed in SEEDS:
        # get the normalized hypervolumes we extracted earlier
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_eum_data = pd.read_csv(file)

        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            # Filter columns that start with "eum"
            col = f"eum/{env}"

            specialist_eum = specialist_eum_data[col].values[0]
            generalist_eum = seed_eum_data[col].values[0]
            
            env_eugr = max(generalist_eum / specialist_eum, 0) # ensure that the EUGR is not negative

            seed_eum_data[f'EUGR/{env}'] = env_eugr
        
        seed_eum_data.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)